## HW4: Significance of Coefficients

### Lucas Baker

Section 3.2 of the HW4 solutions computes the significance of coefficients in a logistic regression model by bootstrapping with $n=100$ samples and examining percentile scores for each feature to generate a 95% confidence interval. If the resulting interval includes zero, the coefficient is judged not to be statistically significant.

Although this method may appear correct, it is in fact modeling the confidence interval for one draw from the true distribution, or the probability that one such sampled coefficient will have the opposite sign from the mean. What we should be measuring is the confidence interval for the true mean of the coefficient, or equivalently, the probability that the true mean is significantly different from zero. Although this can be accomplished with a simple one-sample t-test against a population mean of zero, we can also use the bootstrapping method to manually examine the z-scores for the mean of each coefficient.

The basic difference is as follows:

In [1]:
import numpy as np
from scipy.stats import ttest_1samp, norm

def yesno(b):
    return {True: 'Yes', False: 'No'}[b]

def significant(lower, upper):
    return lower > 0 or upper < 0

# Let the true distribution be normal with a mean and standard deviation of 1.0
bootstrap_coef_samples = np.random.normal(loc=1.0, scale=1.0, size=100)

# Wrong (approach used in solutions)
ci_upper = np.percentile(bootstrap_coef_samples, 97.5)
ci_lower = np.percentile(bootstrap_coef_samples, 2.5)
print("CI range according to bootstrap percentiles: %.2f, %.2f" % (ci_lower, ci_upper))
print("Significant? %s\n" % yesno(significant(ci_lower, ci_upper)))

# Right (Z-score)
boot_mean, boot_std = np.mean(bootstrap_coef_samples), np.std(bootstrap_coef_samples)
boot_stderr = boot_std / np.sqrt(len(bootstrap_coef_samples))
null_mean = 0
truemean_z_score = (boot_mean - null_mean) / boot_stderr
print("Z-score: %.2f" % truemean_z_score)
print("Significant? %s\n" % yesno(abs(truemean_z_score) > 1.96))

# Confirmation (T-test)
tp = ttest_1samp(bootstrap_coef_samples, null_mean)
print("T-score and p-value: %.2f, %.2f" % tp)
print("Significant? %s\n" % yesno(tp[1] < .05))

# Confirmation (CI)
# See https://stackoverflow.com/questions/28242593/correct-way-to-obtain-confidence-interval-with-scipy
datapoint_ci = norm.interval(.95, loc=boot_mean, scale=boot_std)
truemean_ci = norm.interval(.95, loc=boot_mean, scale=boot_stderr)
print("CI for one coefficient: %.2f, %.2f" % datapoint_ci)
print("CI for true mean of the coefficient: %.2f, %.2f" % truemean_ci)

CI range according to bootstrap percentiles: -0.87, 2.38
Significant? No

Z-score: 9.49
Significant? Yes

T-score and p-value: 9.44, 0.00
Significant? Yes

CI for one coefficient: -0.97, 2.80
CI for true mean of the coefficient: 0.73, 1.10


Examining the above, we can see that the percentile method is wrong for two other reasons. Firstly, it produces results essentially equivalent to performing a t-test using the mean and sample standard deviation, when we should be using the mean and sample standard error. The following post illustrates the difference:

https://stackoverflow.com/questions/28242593/correct-way-to-obtain-confidence-interval-with-scipy

The standard deviation is appropriate when assessing the confidence interval for one data point in a sample, while the standard error is appropriate when the objective is to find the confidence interval of the population mean - or, in this case, the mean of the distribution from which our coefficient is being drawn.

Secondly, the percentile method is insensitive to $n$. Consider a coefficient whose true value is normally distributed with $\mu=1.0$ and $\sigma=1.0$. Sample values more than one standard deviation below the mean will fall below zero, so we can expect approximately 16% of sampled coefficients to be negative. Thus, the bootstrapping percentile method will find this coefficient not to be significant, whether we bootstrap $100$, $1000$, or $1$ million times. This is wrong, as our confidence interval for the true mean should narrow with more samples, and as the number of samples approaches infinity we should find this coefficient significant at $p=.05$ or any other threshold.

The code below illustrates that we can fail to find significance even when $n$ is very large.

In [2]:
bootstrap_coef_samples = np.random.normal(loc=1.0, scale=1.0, size=int(10e6))

print("Percent of values less than 0: %.2f" %
      (100 * sum(bootstrap_coef_samples < 0)/len(bootstrap_coef_samples)))
ci_upper = np.percentile(bootstrap_coef_samples, 97.5)
ci_lower = np.percentile(bootstrap_coef_samples, 2.5)
print("Considered significant by bootstrap percentile method? %s" %
      ("Yes" if ci_lower > 0 else "No"))

Percent of values less than 0: 15.83
Considered significant by bootstrap percentile method? No


We should be finding significance, but even with 1 million samples, we do not.

A one-sample t-test fixes the problem, as the CI of a single point remains wide but the CI of the mean converges:

In [3]:
for n_bootstrap in [2, 5, 10, 100, 1000, int(10e6)]:
    sample = bootstrap_coef_samples[:n_bootstrap]
    t, p = ttest_1samp(sample, 0)
    coef_mean, coef_stdev = np.mean(sample), np.std(sample)
    coef_stderr = coef_stdev / np.sqrt(len(sample))
    conf_datapoint = (coef_mean - 1.96 * coef_stdev, coef_mean + 1.96 * coef_stdev)
    conf_truemean = (coef_mean - 1.96 * coef_stderr, coef_mean + 1.96 * coef_stderr)
    print("Confidence interval of a single sampled coefficient: %.2f, %.2f" % conf_datapoint)
    print("Confidence interval of the true coefficient mean: %.2f, %.2f" % conf_truemean)
    print("T-score and p-value for n=%d: %.2f, %.2f" % (n_bootstrap, t, p))
    print("Significant at p = .05? %s\n" % ("Yes" if p < .05 else "No"))

Confidence interval of a single sampled coefficient: -0.51, 3.62
Confidence interval of the true coefficient mean: 0.10, 3.02
T-score and p-value for n=2: 1.48, 0.38
Significant at p = .05? No

Confidence interval of a single sampled coefficient: -1.41, 3.00
Confidence interval of the true coefficient mean: -0.19, 1.78
T-score and p-value for n=5: 1.41, 0.23
Significant at p = .05? No

Confidence interval of a single sampled coefficient: -0.74, 2.73
Confidence interval of the true coefficient mean: 0.44, 1.54
T-score and p-value for n=10: 3.36, 0.01
Significant at p = .05? Yes

Confidence interval of a single sampled coefficient: -0.85, 2.61
Confidence interval of the true coefficient mean: 0.70, 1.05
T-score and p-value for n=100: 9.88, 0.00
Significant at p = .05? Yes

Confidence interval of a single sampled coefficient: -0.98, 3.11
Confidence interval of the true coefficient mean: 1.00, 1.13
T-score and p-value for n=1000: 32.13, 0.00
Significant at p = .05? Yes

Confidence interval